In [1]:
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
import xarray as xr
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt

## 0. Preprocessing Alibaba data

In [386]:
df = pd.read_csv("ecs_series_1_uid_encoded(1).csv")
datetime_object = datetime.datetime.strptime("9/27/18 16:00", "%m/%d/%y %H:%M")
df['Date'] = df["ds"].apply(lambda x:datetime.datetime.strptime(x, "%m/%d/%y %H:%M"))
df = df.drop(columns = ['ds','region_no_factor','iz_no_factor','product_factor','instance_type_name_factor','instance_type_family_factor'])

In [387]:
df = df.groupby(by=['uid','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'uid',values = 'vcpu_net_delead')

## 0.1 Preprocessing cdnow

In [2]:
df = pd.read_csv("CDNOW_master.txt",header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)

In [3]:
df.sample(5)

,id,Date,disks,price_purchase
56974,19011,19980204,2,15.48
53933,17888,19970920,2,15.98
63177,21286,19980121,2,23.98
17010,5379,19970714,4,55.25
50270,16589,19970401,2,25.74


In [4]:
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
12540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.2 Preprocessing data from startup

In [12]:
df_frontendata = pd.read_csv("data/frontendData.csv").drop(columns =['manufacturer','model','operatingSysterm','operatingSystemVersion','timezone','event'])
df_orderdata = pd.read_csv("data/orderdata.csv").set_index("Unnamed: 0").drop(columns = ['supplierId','orderSupplierStatus'])
df_userdata = pd.read_csv("data/userdata.csv").set_index("Unnamed: 0")

In [13]:
df_frontendata['timestamp'] = df_frontendata.dropna()["timestamp"].apply(lambda x:datetime.datetime.strptime(str(x)[:10],"%Y-%m-%d"))
df_frontendata['event']= 1
df_frontendata = df_frontendata.groupby(by=['userid','timestamp']).sum().reset_index()
# If we have several actions per day - it is still 1 event
df_frontendata['event']= 1
df_frontendata.head()

,userid,timestamp,event
0,0043a6cb-6314-4844-bd20-06c41915d398,2018-11-30,1
1,0043a6cb-6314-4844-bd20-06c41915d398,2018-12-03,1
2,0043a6cb-6314-4844-bd20-06c41915d398,2019-01-25,1
3,0043a6cb-6314-4844-bd20-06c41915d398,2019-02-07,1
4,0043a6cb-6314-4844-bd20-06c41915d398,2019-02-13,1


In [14]:
# Extract total amount from list of products
def products_to_amount(s):
    amount = 0 
    #print(s)
    for x in ast.literal_eval(s):
        if 'amount' in x:
            amount += int(x['amount'])
    return amount

In [15]:
df_orderdata['createdAt'] = df_orderdata.dropna()["createdAt"].apply(lambda x:datetime.datetime.utcfromtimestamp(int(str(x)[:10])))
df_orderdata['createdAt'] = df_orderdata.dropna()["createdAt"].apply(lambda x:datetime.datetime.strptime(str(x)[:10],"%Y-%m-%d"))
df_orderdata['products'] = df_orderdata.dropna()["products"].apply(lambda x:products_to_amount(x))
df_orderdata.columns = ['timestamp','amount','userid']
df_orderdata= df_orderdata.reset_index().drop(columns = ['Unnamed: 0'])
df_orderdata = df_orderdata.groupby(by=['userid','timestamp']).sum().reset_index()
df_orderdata.head()

,userid,timestamp,amount
0,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-02,19.0
1,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-16,8.0
2,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-23,19.0
3,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-30,16.0
4,00653016-d177-4080-86e6-4c69b865cb02,2019-01-27,147.0


In [16]:
df_front_back_combined = df_frontendata.merge(df_orderdata,how = 'outer',left_on =['timestamp','userid'],right_on=['timestamp','userid']).dropna(subset=['timestamp', 'userid'])
#df_front_back_combined=df_front_back_combined.groupby(by=['userid','timestamp']).sum().reset_index()
# For Monetary only amount is needed
# We count for event if either purchase or action took place
df_front_back_combined['event'] = df_front_back_combined.apply((lambda x: 1 if (np.isnan(x['amount']) or np.isnan(x['event'])) else 2) ,axis =1 )
df_front_back_combined=df_front_back_combined.groupby(by=['userid','timestamp']).sum().reset_index()
df_front_back_combined.sample(5)

,userid,timestamp,event,amount
26167,c5db0ead-3624-4eb8-9ba5-49a1f7798597,2018-10-05,1,130.0
19970,96810624-ed3b-463a-89d3-ec9b25d3ce5e,2018-08-27,1,170.0
32849,f3bae647-c991-4e66-8c9d-d9b8723dba7c,2019-01-23,2,634.0
4923,2cab3b99-309e-410c-bf25-3883d843c1e4,2019-01-02,2,35.0
16203,7ace89bd-55a3-4d3f-9d82-88d26631cb77,2019-03-03,2,32.0


### 0.2.9 Taking random 2000 users

In [17]:
df__time_series_events = df_front_back_combined.pivot(columns = 'timestamp',index = 'userid',values = 'event')
df__time_series_amount = df_front_back_combined.pivot(columns = 'timestamp',index = 'userid',values = 'amount')
df__time_series_events.to_csv('export/startup/time_series_events.csv')
df__time_series_amount.to_csv('export/startup/time_series_amount.csv')

## 0.3 Preprocessing bimbo

In [8]:
df = pd.read_csv("bimbo_2000.csv")
df = df.drop(columns = ['Unnamed: 0'])
df = df.groupby(by=['uid','week']).sum().reset_index()
df = df.pivot(columns = 'week',index = 'uid',values = 'target')
df.to_csv('export/bimbo/bimbo_initial_time_series_2000.csv')
df.sample(5)

week,1,2,3,4,5,6,7
uid,,,,,,,
2477407,21.0,2.0,20.0,88.0,44.0,0.0,0.0
2390042,20.0,3.0,1.0,1.0,1.0,17.0,6.0
4687900,0.0,6.0,28.0,9.0,4.0,20.0,14.0
2290023,10.0,0.0,0.0,0.0,0.0,0.0,0.0
4399356,112.0,55.0,45.0,34.0,23.0,33.0,32.0


## 0.9. Taking random 2000 users

In [5]:
# Skip for Startup Data
df_test = df.sample(2000)
df_test.to_csv('cdnow_time_series_initial_2000.csv')
df_test.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
8654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.0 RFM-analysis for alibaba data

In [318]:
df_recency =  pd.DataFrame(columns = df.columns)
df_monetary =  pd.DataFrame(columns = df.columns)
df_frequency = pd.DataFrame(columns = df.columns)

In [74]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df_test.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        #since first event
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    first_purchase = -1#flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_days_of_purchases/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

## 1.1 Modified RFM-analysis for startup data

In [27]:
df_recency =  pd.DataFrame(columns = df__time_series_events.columns)
df_monetary =  pd.DataFrame(columns = df__time_series_amount.columns)
df_frequency = pd.DataFrame(columns = df__time_series_events.columns)

In [28]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df__time_series_events.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry) or entry==0):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        # since first purchase
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

for index, row in df__time_series_amount.iterrows():
    arr = []
    arr = list(row)
    mon_arr = []
    #MONETARY  Moving average monetary value
    total_amount = 0 
    first_purchase = -1 #flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry) or entry==0):
            total_amount = total_amount
        else:
            total_amount+=int(entry)
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_amount/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr


## 1.2 Modified RFM-analysis for BIMBO and cdnow data

In [6]:
df_recency =  pd.DataFrame(columns = df_test.columns)
df_monetary =  pd.DataFrame(columns = df_test.columns)
df_frequency = pd.DataFrame(columns = df_test.columns)

In [7]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df_test.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    first_purchase = -1#flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_days_of_purchases/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

## 2. Clustering

In [29]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

Resumed because of empty cluster
Resumed because of empty cluster
0.215 --> 0.212 --> 0.212 --> 0.212 --> 0.211 --> 0.210 --> 0.210 --> 0.210 --> 
0.050 --> 0.049 --> 0.049 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 0.048 --> 
0.153 --> 0.092 --> 0.083 --> 0.082 --> 0.083 --> 


In [30]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [31]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [32]:
df_monetary.to_csv("export/startup/startup_monetary.csv")
df_frequency.to_csv("export/startup/startup_frequency.csv")
df_recency.to_csv("export/startup/startup_recency.csv")
df_centroids.to_csv("export/startup/startup_cluster_centroids.csv")